# Jensen-Shannon Distance vs Layers and Theta

### Overview
This task explores how quantum noise affects the accuracy of the quantum Galton Box under varying circuit depths (layers) and coin-bias angles (theta). The Jensen-Shannon Distance (JSD) is used to quantify the divergence between ideal and noisy distributions.

### Goals:
 - Simulate quantum circuits under noise and ideal conditions
 - Compare their output probability distributions
 - Compute Jensen-Shannon Distance (JSD) as a similarity metric
 - Visualize how noise affects results as a function of layer count and theta angle


In [ ]:
# Imports
# ---------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error

import os
# Create output directory
output_dir = "../outputs/Derivable4e/"
os.makedirs(output_dir, exist_ok=True)  # Create if doesn't exist

# ---------------------------------------------------
# Utility Function: Jensen-Shannon Distance
# ---------------------------------------------------
def jensen_shannon_distance(p, q):
    """
    Compute the Jensen-Shannon Distance between two probability distributions.
    
    Parameters:
        p (np.ndarray): First probability distribution.
        q (np.ndarray): Second probability distribution.
    
    Returns:
        float: The Jensen-Shannon distance.
    """
    p = np.array(p, dtype=float)
    q = np.array(q, dtype=float)
    p /= p.sum()
    q /= q.sum()
    m = 0.5 * (p + q)
    return np.sqrt(0.5 * (entropy(p, m) + entropy(q, m)))

# ---------------------------------------------------
# Core Function: Build the quantum Galton circuit
# ---------------------------------------------------
def build_quantum_galton_box(n_layers, theta=np.pi/4):
    """
    Constructs a quantum Galton box circuit with a parameterized coin rotation.
    
    Parameters:
        n_layers (int): Number of layers in the Galton board.
        theta (float): Angle for Ry gate to simulate a biased coin.
        
    Returns:
        QuantumCircuit: The constructed quantum circuit.
    """
    coin = QuantumRegister(1, 'coin')
    pos = QuantumRegister(n_layers, 'pos')
    classical = ClassicalRegister(n_layers, 'c')
    qc = QuantumCircuit(coin, pos, classical)

    for i in range(n_layers):
        qc.ry(2 * theta, coin[0])
        qc.cx(coin[0], pos[i])
        if i < n_layers - 1:
            qc.swap(coin[0], pos[i])

    qc.measure(pos, classical)
    return qc

# ---------------------------------------------------
# Function: Create a basic noise model
# ---------------------------------------------------
def create_noise_model(p_depol=0.01, t1=50e3, t2=70e3, gate_time=100):
    """
    Builds a basic noise model using depolarizing and thermal relaxation errors.
    
    Parameters:
        p_depol (float): Depolarizing probability.
        t1 (float): T1 relaxation time (ns).
        t2 (float): T2 relaxation time (ns).
        gate_time (int): Gate execution time (ns).
    
    Returns:
        NoiseModel: Qiskit noise model.
    """
    noise_model = NoiseModel()

    dep_err_1 = depolarizing_error(p_depol, 1)
    dep_err_2 = depolarizing_error(p_depol, 2)

    therm_err_1 = thermal_relaxation_error(t1, t2, gate_time)
    therm_err_2 = thermal_relaxation_error(t1, t2, gate_time).tensor(
        thermal_relaxation_error(t1, t2, gate_time))

    noise_model.add_all_qubit_quantum_error(dep_err_1, ['ry'])
    noise_model.add_all_qubit_quantum_error(dep_err_2, ['cx'])
    noise_model.add_all_qubit_quantum_error(therm_err_1, ['ry'])
    noise_model.add_all_qubit_quantum_error(therm_err_2, ['cx'])

    return noise_model

# ---------------------------------------------------
# Simulation & JSD Computation
# ---------------------------------------------------
def run_jsd_experiment():
    """
    Runs simulations across varying layers and theta values, computes the Jensen-Shannon
    distance between ideal and noisy simulations, and plots the results.
    """
    shots = 2048
    layers_range = range(2, 9)
    theta_range = np.linspace(0.1, np.pi/2, 10)
    
    noise_model = create_noise_model()
    simulator = AerSimulator()
    simulator_noise = AerSimulator(noise_model=noise_model)

    jsd_matrix = np.zeros((len(layers_range), len(theta_range)))

    for i, layers in enumerate(layers_range):
        for j, theta in enumerate(theta_range):
            # Build and transpile circuit
            qc = build_quantum_galton_box(layers, theta)
            tqc = transpile(qc, simulator)
            tqc_noisy = transpile(qc, simulator_noise)

            # Run ideal simulation
            ideal_counts = simulator.run(tqc, shots=shots).result().get_counts()
            ideal_probs = np.zeros(2**layers)
            for bitstring, count in ideal_counts.items():
                index = int(bitstring, 2)
                ideal_probs[index] = count / shots

            # Run noisy simulation
            noisy_counts = simulator_noise.run(tqc_noisy, shots=shots).result().get_counts()
            noisy_probs = np.zeros(2**layers)
            for bitstring, count in noisy_counts.items():
                index = int(bitstring, 2)
                noisy_probs[index] = count / shots

            # Compute JSD
            jsd = jensen_shannon_distance(ideal_probs, noisy_probs)
            jsd_matrix[i, j] = jsd
            print(f"Layers: {layers}, Theta: {theta:.2f} → JSD: {jsd:.4f}")

    # Plot heatmap
    plt.figure(figsize=(12, 6))
    sns.heatmap(jsd_matrix, xticklabels=[f"{t:.2f}" for t in theta_range],
                yticklabels=list(layers_range), cmap="viridis", annot=True, fmt=".2f")
    plt.xlabel("Theta (coin bias)")
    plt.ylabel("Number of Layers")
    plt.title("Jensen-Shannon Distance (Noise vs Ideal)")
    plt.tight_layout()
    plot_path = os.path.join(output_dir, "jensen-shannon_distance.png")
    plt.savefig(plot_path, dpi=300)
    print(f"[✓] Plot saved to: {plot_path}")
    plt.show()

# ---------------------------------------------------
# Run the full experiment
# ---------------------------------------------------
run_jsd_experiment()


Layers: 2, Theta: 0.10 → JSD: 0.0547
Layers: 2, Theta: 0.26 → JSD: 0.0258
Layers: 2, Theta: 0.43 → JSD: 0.0263
Layers: 2, Theta: 0.59 → JSD: 0.0249
Layers: 2, Theta: 0.75 → JSD: 0.0140
Layers: 2, Theta: 0.92 → JSD: 0.0121
Layers: 2, Theta: 1.08 → JSD: 0.0259
Layers: 2, Theta: 1.24 → JSD: 0.0334
Layers: 2, Theta: 1.41 → JSD: 0.0587
Layers: 2, Theta: 1.57 → JSD: 0.1022
Layers: 3, Theta: 0.10 → JSD: 0.0692
Layers: 3, Theta: 0.26 → JSD: 0.0334
Layers: 3, Theta: 0.43 → JSD: 0.0266
